In [1]:
import json
import time
import re
import urllib
from datetime import date
import polars as pl
import numpy as np
import textdistance as td
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tobias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
schema={
    'id': pl.Utf8,
    'abstract': pl.Utf8,
    'beratungsstand': pl.Utf8,
    'vorgangstyp': pl.Utf8,
    'gesta': pl.Utf8,
    'archiv': pl.Utf8,
    'sachgebiet': pl.List(pl.Utf8),
    'typ': pl.Utf8,
    'wahlperiode': pl.UInt64,
    'zustimmungsbeduerftigkeit': pl.List(pl.Utf8),
    'initiative': pl.List(pl.Utf8),
    'deskriptor': pl.List(pl.Struct([
        pl.Field('fundstelle', pl.Boolean),
        pl.Field('name', pl.Utf8),
        pl.Field('typ', pl.Utf8),
    ])),
    'aktualisiert': pl.Utf8,
    'titel': pl.Utf8,
    'datum': pl.Utf8,
    'verkuendung': pl.List(pl.Struct([
        pl.Field('verkuendungsdatum', pl.Utf8)
    ]))
}
df_vorgang = pl.DataFrame(json.load(open('vorgang.json', encoding='utf-8')), schema=schema)
df_gesetze = df_vorgang.filter(pl.col('vorgangstyp') == 'Gesetzgebung')
pl.Config(fmt_str_lengths=100)
df_gesetze['beratungsstand'].unique()

beratungsstand
str
"""Bundesrat hat Vermittlungsausschuss nicht angerufen"""
"""Abgelehnt"""
"""Abgeschlossen - Ergebnis siehe Vorgangsablauf"""
"""Noch nicht beraten"""
"""Bundesrat hat zugestimmt"""
"""Dem Bundestag zugeleitet - Noch nicht beraten"""
"""Verkündet"""
"""Beschlussempfehlung liegt vor"""
"""Bundesrat hat Zustimmung versagt"""


In [3]:
df_vorgangspos = pl.read_json('vorgangsposition.json')
df_vorgangspos = df_vorgangspos.with_columns(
   pl.col("datum").str.to_date(format="%Y-%m-%d")
)
df_vorgangspos_gesetzgebung = df_vorgangspos.filter((pl.col('vorgangstyp') == 'Gesetzgebung') & (pl.col('typ') == 'Vorgangsposition'))
df_vorgang_duration = df_vorgangspos_gesetzgebung.group_by(['vorgang_id']).agg([
    pl.col('datum').min().alias('vorgang_start'), 
])

In [4]:
df_gesetze.head()

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]]
"""308708""","""Maßnahmen zur Steigerung der Attraktivität der beruflichen Bildung: Einführung eines Validierungsve…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""K007""",null,"[""Arbeit und Beschäftigung"", ""Bildung und Erziehung"", ""Medien, Kommunikation und Informationstechnik""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 73/24)""]","[""Bundesregierung""]","[{false,""Ausbildungsvertrag"",""Sachbegriffe""}, {false,""Behinderter"",""Sachbegriffe""}, … {false,""Schulzeugnis"",""Sachbegriffe""}]","""2024-03-27T17:08:03+01:00""","""Berufsbildungsvalidierungs- und -digitalisierungsgesetz (BVaDiG)""","""2024-03-27""",null
"""302813""","""Ermöglichung massiver und zügiger Investitionen in Modernisierung und Ausbau von Eisenbahninfrastru…","""Im Vermittlungsverfahren""","""Gesetzgebung""","""J015""",null,"[""Verkehr"", ""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 379/23 (bes.eilbed.))""]","[""Bundesregierung""]","[{false,""Bahnstrecke"",""Sachbegriffe""}, {false,""Bahntechnik"",""Sachbegriffe""}, … {false,""Wettbewerb"",""Sachbegriffe""}]","""2024-03-27T09:36:30+01:00""","""Viertes Gesetz zur Änderung des Bundesschienenwegeausbaugesetzes""","""2024-03-26""",null
"""300144""",null,"""Bundesrat hat Zustimmung versagt""","""Gesetzgebung""","""B039""",null,"[""Medien, Kommunikation und Informationstechnik"", ""Staat und Verwaltung""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 226/23)""]","[""Bundesregierung""]","[{false,""Abgabenordnung"",""Rechtsmaterialien""}, {false,""Barrierefreiheit"",""Sachbegriffe""}, … {false,""Öffentliche Verwaltung"",""Sachbegriffe""}]","""2024-03-27T09:30:43+01:00""","""Gesetz zur Änderung des Onlinezugangsgesetzes sowie weiterer Vorschriften zur Digitalisierung der V…","""2024-03-26""",null
"""308776""","""Zustimmung zu dem am 21. Juli 2023 in Lauterbourg unterzeichneten Abkommen: Standardisierung grenzü…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""XK002""",null,"[""Arbeit und Beschäftigung"", ""Bildung und Erziehung""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 76/24)""]","[""Bundesregierung""]","[{false,""Ausbildungsvertrag"",""Sachbegriffe""}, {false,""Auswärtige Kultur- und Bildungspolitik"",""Sachbegriffe""}, … {false,""Teilzeitarbeit"",""Sachbegriffe""}]","""2024-03-26T13:54:05+01:00""","""Gesetz zu dem Abkommen vom 21. Juli 2023 zwischen der Regierung der Bundesrepublik Deutschland und …","""2024-03-22""",null
"""308709""","""Zusammenfassung und anwendungsbezogene Konkretisierung der sich aus der Anwendung des BEPS-MLI erge…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""D057""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 75/24)""]","[""Bundesregierung""]","[{true,""BEPS-MLI-Anwendungsgesetz"",""Rechtsmaterialien""}, {false,""Bundeszentralamt für Steuern"",""Institutionen""}, … {false,""Unternehmensbesteuerung"",""Sachbegriffe""}]","""2024-03-26T13:53:58+01:00""","""Gesetz zur Anwendung des Mehrseitigen Übereinkommens vom 24. November 2016 und zu weiteren Maßnahme…","""2024-03-22""",null


In [5]:
df_gesetze = df_gesetze.join(df_vorgang_duration.select(['vorgang_id', 'vorgang_start']), left_on='id', right_on='vorgang_id', how='left')

In [6]:
#df_vorgang_docs = df_vorgangspos.filter(pl.col('dokumentart') == 'Drucksache').group_by('vorgang_id').agg(pl.col('fundstelle').struct.field("dokumentnummer").explode())
#df_gesetze = df_gesetze.join(df_vorgang_docs.select(['vorgang_id', 'dokumentnummer']), left_on='id', right_on='vorgang_id', how='left')

In [7]:
today = date.today()
df_gesetze = df_gesetze.with_columns(
    pl.when(pl.col('beratungsstand') == 'Verkündet').then(
        pl.col('verkuendung').list.eval(pl.col('').struct.field('verkuendungsdatum').str.to_date(format="%Y-%m-%d")).list.min()
    ).when(pl.col('beratungsstand') == 'Abgelehnt').then(
        pl.col('datum').str.to_date(format="%Y-%m-%d")
    ).otherwise(today).alias('vorgang_end')
)
df_gesetze = df_gesetze.with_columns((pl.col('vorgang_end') - pl.col('vorgang_start')).dt.total_days().alias('vorgangsdauer'))

In [8]:
reg_frag = ['Fraktion BÜNDNIS 90/DIE GRÜNEN', 'Fraktion der FDP', 'Fraktion der SPD']

def fix_initiative(x):
    return pl.when(x.list.set_intersection(reg_frag).len == len(reg_frag)).then(
        x.list.set_difference(reg_frag)
    ).otherwise(x)

df_gesetze = df_gesetze.with_columns(pl.when(pl.col('initiative').list.set_intersection(reg_frag).list.len() == len(reg_frag)).then(
        pl.col('initiative').list.set_difference(reg_frag).list.concat(pl.Series(['Bundesregierung']))
    ).otherwise(pl.col('initiative')).alias('initiative'))

In [9]:
df_gesetze = df_gesetze.with_columns(pl.col('deskriptor').list.eval(pl.element().struct.field('name')).alias('keywords'))

In [10]:
df_gesetze.select(pl.col("*").exclude("abstract", "vorgangstyp", "gesta", "archiv", "typ", "deskriptor")).write_json('gesetze.json', row_oriented=True)

In [11]:
df_gesetze_public = df_gesetze.filter(pl.col('beratungsstand') == 'Verkündet')
bins = [7, 14, 30, 60, 120, 240, 365]
duration_bin = np.digitize(df_gesetze_public.select('vorgangsdauer').to_series(), bins, right=True)
df_gesetze_public.select('vorgangsdauer').to_series().hist(bins)

break_point,category,vorgangsdauer_count
f64,cat,u32
7.0,"""(-inf, 7.0]""",1
14.0,"""(7.0, 14.0]""",3
30.0,"""(14.0, 30.0]""",19
60.0,"""(30.0, 60.0]""",22
120.0,"""(60.0, 120.0]""",71
240.0,"""(120.0, 240.0]""",109
365.0,"""(240.0, 365.0]""",5
inf,"""(365.0, inf]""",1


In [12]:
df_gesetze_public_bins = df_gesetze_public.with_columns(pl.from_numpy(duration_bin, schema=['duration_bin']))
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').min())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""283063""","""Impfpflicht gegen COVID-19 bzw. Nachweis über Genesung oder Kontraindikation für Beschäftigte besti…","""Verkündet""","""Gesetzgebung""","""M002""","""XX/2""","[""Gesundheit""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 20/188)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Apotheker"",""Sachbegriffe""}, {false,""Arbeitnehmer-Entsendegesetz"",""Rechtsmaterialien""}, … {false,""Zahnarzt"",""Sachbegriffe""}]","""2022-07-26T19:57:25+02:00""","""Gesetz zur Stärkung der Impfprävention gegen COVID-19 und zur Änderung weiterer Vorschriften im Zus…","""2021-12-10""","[{""2021-12-11""}]",2021-12-06,2021-12-11,5,"[""Apotheker"", ""Arbeitnehmer-Entsendegesetz"", … ""Zahnarzt""]",0


In [13]:
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').max())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""297317""",null,"""Verkündet""","""Gesetzgebung""","""XB002""","""XX/225""","[""Europapolitik und Europäische Union""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 104/23)"", ""Ja, laut Verkündung (BGBl II)""]","[""Bundesregierung""]","[{false,""Europawahl"",""Sachbegriffe""}, {false,""Europäisches Parlament"",""Institutionen""}, {false,""Sperrklausel"",""Sachbegriffe""}]","""2024-03-13T16:36:41+01:00""","""Gesetz zu dem Beschluss (EU, Euratom) 2018/994 des Rates der Europäischen Union vom 13. Juli 2018 z…","""2023-07-07""","[{""2024-03-13""}]",2023-03-10,2024-03-13,369,"[""Europawahl"", ""Europäisches Parlament"", ""Sperrklausel""]",7


In [14]:
def map_gesetze(tokens):
    overlap = df_gesetze.with_columns(pl.col('titel.stem').map_elements(lambda x: td.lcsstr.normalized_similarity(tokens, x)).alias('similarity')).select(['titel', 'titel.stem', 'id', 'similarity'])
    overlap_ = overlap.sort('similarity', descending=True)
    overlap = overlap_.filter(pl.col('similarity') > 0.4)
    if overlap.shape[0] == 0:
        print(overlap_.head())
        raise RuntimeError('Count not find any Gesetze with similarity > 0.4')
    if overlap.shape[0] > 1:
        if overlap.row(1, named=True)['similarity'] * 2 > overlap.row(0, named=True)['similarity']:
            print(f'Error: could not map gesetz: {tokens}')
            print(overlap.head())
            raise RuntimeError
    #print(overlap.head())
    return overlap.row(0, named=True)

bad_words = [
    'beratung',
    'erst',
    'zweit',
    'dritt',
    'gebrach',
    'entwurf',
    'eine'
]
stopwords_german = stopwords.words('german')
def rm_stop_words(txt):
    words = txt.split()
    words = list(filter(lambda x: x not in stopwords_german, words))
    return ' '.join(words)
    
def pre_process(txt):
    txt = ' '.join(txt.split())
    txt = re.sub(r'[a-z]\)', '', txt)
    txt = re.sub(r'des von der .*(fraktio|bundesregierung)', '', txt)
    txt = re.sub(r'^–', '', txt)
    for word in bad_words:
        txt = txt.replace(word, '', 1)
    return rm_stop_words(txt)

stemmer = Cistem()
def stem(txt):
    words = []
    for word in txt.split():
        words.append(stemmer.segment(word)[0])
    return ' '.join(words)

def map_top_gesetze(soup):
    tagesordnung = {}
    for conf in soup.find_all('div', {'class': 'bt-conference-title'}):
        sitzung = conf.text.strip()
        print(sitzung)
        top_l = []
        for thema in conf.parent.parent.find_all('td', {'data-th': 'Thema'}):
            if thema.p:
                for top in str(thema.p).split('<br/><br/>'):
                    top = BeautifulSoup(top, 'lxml')
                    vorgang_id = None
                    for a in top.find_all('a'):
                        drucksache = df_vorgangspos.filter((pl.col('dokumentart') == 'Drucksache') & (pl.col('fundstelle').struct.field("drucksachetyp") == 'Gesetzentwurf') & (pl.col('fundstelle').struct.field("pdf_url") == a['href']))
                        if drucksache.shape[0] == 0:
                            continue
                        if drucksache.shape[0] == 1:
                            vorgang_id_new = drucksache.row(0, named=True)['vorgang_id']
                            if vorgang_id and vorgang_id != vorgang_id_new:
                                raise RuntimeError('vorgang id missmatch')
                            vorgang_id = vorgang_id_new
                        if drucksache.shape[0] > 1:
                            print(drucksache)
                            raise RuntimeError('Found multiple vorgang ids for document')
                    if vorgang_id:
                        gesetz = df_gesetze.filter(pl.col('id') == vorgang_id).row(0, named=True)
                        print('* ' + gesetz['titel'])
                        top_l.append(gesetz)
                    else:
                        txt_ = top.get_text(' ')
                        for txt in txt_.split('Drucksache'):
                            if 'gesetz' in txt.lower() and 'entwurf' in txt.lower():
                                txt = stem(txt)
                                txt = pre_process(txt)
                                gesetz = map_gesetze(txt)
                                print('* ' + gesetz['titel'])
                                top_l.append(gesetz)
        print()
        tagesordnung[sitzung] = top_l
    return tagesordnung

def get_tagesordnung_gesetze(kw):
    res = requests.get('https://www.bundestag.de/apps/plenar/plenar/conferenceweekDetail.form', {'year': 2024, 'week': kw, 'limit': 10})
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'lxml')
    return map_top_gesetze(soup)

In [15]:
df_gesetze = df_gesetze.with_columns(pl.col('titel').map_elements(stem).map_elements(rm_stop_words).alias('titel.stem'))

In [16]:
#df_vorgangspos.filter((pl.col('dokumentart') == 'Drucksache')).select(pl.col('fundstelle').struct.field("pdf_url"))
#df_vorgangspos.filter(pl.col('fundstelle').struct.field("pdf_url").eq('https://dserver.bundestag.de/btd/20/106/2010664.pdf'))

In [17]:
res = requests.get('https://www.bundestag.de/tagesordnung')
soup = BeautifulSoup(res.text, 'lxml')
div = soup.find('div', {'class': 'bt-module-row bt-module-row-sitzungsablauf'})
url = urllib.parse.urlparse(div['data-dataloader-url'])
week = int(urllib.parse.parse_qs(url.query)['week'][0])

top = {}
for prev in range(1, 10):
    tagesordnung = get_tagesordnung_gesetze(week - prev)
    if tagesordnung:
        top.update(tagesordnung)
        break
top.update(get_tagesordnung_gesetze(week))
for prev in range(1, 10):
    tagesordnung = get_tagesordnung_gesetze(week + prev)
    if tagesordnung:
        top.update(tagesordnung)
        break

20. März 2024 (159. Sitzung)

21. März 2024 (160. Sitzung)
* Gesetz zur Durchführung der Verordnung (EU) 2022/2065 des Europäischen Parlaments und des Rates vom 19. Oktober 2022 über einen Binnenmarkt für digitale Dienste und zur Änderung der Richtlinie 2000/31/EG sowie zur Durchführung der Verordnung (EU) 2019/1150 des Europäischen Parlaments und des Rates vom 20. Juni 2019 zur Förderung von Fairness und Transparenz für gewerbliche Nutzer von Online-Vermittlungsdiensten und zur Änderung weiterer Gesetze
* Gesetz zur rechtssicheren Einführung einer Bezahlkarte im Asylbewerberleistungsgesetz (Bezahlkartengesetz - BezahlkG)
* Zweites Gesetz zur Änderung des Umweltstatistikgesetzes, zur Anpassung bestimmter Vorschriften über den Schutz geografischer Herkunftsangaben im Landwirtschaftsbereich und zur Änderung des Bundesnaturschutzgesetzes
* Drittes Gesetz zur Änderung des Bundesschuldenwesengesetzes
* Gesetz zu dem Übereinkommen vom 12. März 2019 zur Gründung des "Square Kilometre Array"-O

In [18]:
json.dump({day: [gesetz['id'] for gesetz in value] for day, value in top.items()}, open('tagesordnung.json', 'wt'))

In [19]:
pre_process(stem('Erste Beratung des von der CDU/CSU-Fraktion eingebrachten Gesetzentwurfs zur rechtssicheren Einführung '))

'gesetz rechtssich einführung'

In [20]:
pre_process(stem(' – Zweite und dritte Beratung des von der Bundesregierung eingebrachten Entwurfs eines Gesetzes zur Durchführung'))

'gesetz durchführung'

In [21]:
pre_process(stem('a) Zweite und dritte Beratung des von der Bundesregierung eingebrachten Entwurfs eines Zweiten Gesetzes zur Änderung des Umweltstatistikgesetzes  '))

'zweit gesetz änderung umweltstatistikgesetz'